# Data Cleaning and Visualization

There are 23 columns and 145460 rows in this time-series dataset. The goal is to predict the `RainTomorrow` variable.

In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# helper function to get quick row info
def getDfRowInfo(dataframe):
    print('DF Row Info:')
    print('\tTotal number of rows:', len(df.index))
    print('\tPercentage of original number of rows:', len(df.index) / 145460)
    print('\tTotal number of rows with the RainTomorrow = true:', len([x for x in df['RainTomorrow'] if x == 'Yes']))
    print('\tPercentage of dataset that has RainTomorrow = true:', len([x for x in df['RainTomorrow'] if x == 'Yes']) / len(df.index))
    print('*********************************************************************************************')

# -----------------------
# read in dataset with date column parsed
df = pd.read_csv('weatherAUS.csv',
    parse_dates=['Date'],
    index_col='Date')

# preview data
getDfRowInfo(df)

# print out the number of non-NA values per column
print('Total number of non-NA values per column, out of', len(df), 'total columns:')
print(df.count(0))


DF Row Info:
	Total number of rows: 145460
	Percentage of original number of rows: 1.0
	Total number of rows with the RainTomorrow = true: 31877
	Percentage of dataset that has RainTomorrow = true: 0.21914615701911178
*********************************************************************************************
Total number of non-NA values per column, out of 145460 total columns:
Location         145460
MinTemp          143975
MaxTemp          144199
Rainfall         142199
Evaporation       82670
Sunshine          75625
WindGustDir      135134
WindGustSpeed    135197
WindDir9am       134894
WindDir3pm       141232
WindSpeed9am     143693
WindSpeed3pm     142398
Humidity9am      142806
Humidity3pm      140953
Pressure9am      130395
Pressure3pm      130432
Cloud9am          89572
Cloud3pm          86102
Temp9am          143693
Temp3pm          141851
RainToday        142199
RainTomorrow     142193
dtype: int64


In [20]:
# NA value management
# Only use rows of df that do not have NA in Rain Tomorrow, the label to predict
df = df[df['RainTomorrow'].notna()]

getDfRowInfo(df)

# replace numerical columns with mean and categorical columns with mode
for column in ['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 
               'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 
               'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 
               'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm']:
    df[column] = df[column].fillna(df[column].mean())

for column in ['WindGustDir', 'WindDir9am', 'WindDir3pm']:
    df[column] = df[column].fillna(df[column].mode())
    
# ------------------------
# transform binary string to binary 0 or 1 values
df['RainToday'] = df['RainToday'].transform(lambda x: 0 if x == 'No' else 1)
df['RainTomorrow'] = df['RainTomorrow'].transform(lambda x: 0 if x == 'No' else 1)

# -------------------------
# TODO: should we split dataset by location?

# transform WindDir and Location columns into encoded labels
la = LabelEncoder()
l = []
for i in df.columns:
    if df.dtypes[i]=='O':
        l.append(i)
        df[i] = la.fit_transform(df[i])
print('Transformed columns to encoded labels: ', l)

# -------------------------
# preview data
df.head()
print(df.describe())

DF Row Info:
	Total number of rows: 142193
	Percentage of original number of rows: 0.9775402172418535
	Total number of rows with the RainTomorrow = true: 31877
	Percentage of dataset that has RainTomorrow = true: 0.22418121848473554
*********************************************************************************************
Transformed columns to encoded labels:  ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
            Location        MinTemp        MaxTemp       Rainfall  \
count  142193.000000  142193.000000  142193.000000  142193.000000   
mean       23.740655      12.186400      23.226784       2.349974   
std        14.237503       6.388924       7.109554       8.423217   
min         0.000000      -8.500000      -4.800000       0.000000   
25%        11.000000       7.600000      17.900000       0.000000   
50%        24.000000      12.000000      22.700000       0.000000   
75%        36.000000      16.800000      28.200000       0.800000   
max        48.000000     

In [3]:
# write cleaned dataset to new file

df.to_csv('cleanedWeatherAUS.csv')

In [4]:
# do some initial analysis? data visualizations on the dataset content
# useful syntax to slice data based on time:
df["1/1/2010":"1/2/2010"].head() # gets all data in the given date range
# min, max temp over time, etc...
# - Valeria

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-01,2,19.4,31.9,5.0,5.468232,7.611178,5,39.0,7,14,...,70.0,40.0,1012.2,1008.5,4.447461,1.00000,23.4,30.9,1,1
2010-01-02,2,18.6,29.1,12.4,5.468232,7.611178,13,56.0,8,13,...,88.0,48.0,1007.8,1006.2,5.000000,4.50993,20.6,28.0,1,0
2010-01-01,4,19.6,29.1,0.0,5.468232,7.611178,6,43.0,5,3,...,83.0,56.0,1014.2,1010.6,4.447461,4.50993,22.4,28.2,0,0
2010-01-02,4,20.3,30.3,0.0,5.468232,7.611178,9,41.0,5,4,...,72.0,64.0,1007.9,1005.5,4.447461,4.50993,24.5,28.6,0,1
2010-01-01,10,19.3,26.4,1.6,4.000000,2.200000,4,30.0,5,5,...,96.0,86.0,1011.1,1008.9,8.000000,8.00000,21.3,23.8,1,1
